Runs a shape regularization CNN training based on UNet

In [1]:
import numpy as np 
np.random.seed(0)  # for reproducibility

import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import EarlyStopping  
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as keras

ModuleNotFoundError: No module named 'skimage'

Shape regularization model (the model is U-Net without concatenation):

In [2]:
def unet():
    n = 64
    inputs = Input((n,n,1))
    conv1 = Conv2D(n//4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(n//4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(n//2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(n//2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(n, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(n, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(2*n, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(2*n, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(4*n, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(4*n, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(2*n, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(2*n, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(2*n, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(n, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    conv7 = Conv2D(n, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(up7)
    conv7 = Conv2D(n, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(n//2, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    conv8 = Conv2D(n//2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(up8)
    conv8 = Conv2D(n//2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(n//4, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    conv9 = Conv2D(n//4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(up9)
    conv9 = Conv2D(n//4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['mae'])
    
    model.summary()

    return model

model = unet()

NameError: name 'Input' is not defined

Train data load and data augmentation

In [20]:
X_train = np.load("X_train.npy")
Y_train = np.load("Y_train.npy")
Y_train = Y_train.reshape(Y_train.shape[0],64,64,1)
print (X_train.shape)
print (Y_train.shape)

data_gen_args_image = dict(featurewise_center=True,
                 featurewise_std_normalization=True,
                 rotation_range=60.,
                 width_shift_range=0.1,
                 height_shift_range=0.1,
                 zoom_range=0)
    
data_gen_args_mask = dict(featurewise_center=False,
                 featurewise_std_normalization=False,
                 rotation_range=60.,
                 width_shift_range=0.1,
                 height_shift_range=0.1,
                 zoom_range=0)
image_datagen = ImageDataGenerator(**data_gen_args_image)
mask_datagen = ImageDataGenerator(**data_gen_args_mask)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(X_train, augment=True, seed=seed)
mask_datagen.fit(Y_train, augment=True, seed=seed)
image_generator = image_datagen.flow(X_train,seed=seed)
mask_generator = mask_datagen.flow(Y_train,seed=seed)
train_generator = zip(image_generator, mask_generator)

(14954, 64, 64, 1)
(14954, 64, 64, 1)


Define keras callbacks

In [29]:

early_stop = EarlyStopping(monitor='mean_absolute_error', min_delta=0.001, patience=5, mode='min', verbose=1)
checkpoint = ModelCheckpoint("weights.{epoch:02d}.hdf5", monitor='mean_absolute_error', verbose=1, save_best_only=False, mode='min', period=1)


Training

In [30]:
model.fit_generator(train_generator,steps_per_epoch=2000,epochs=100,callbacks = [early_stop,checkpoint])
model.save('seg_model.h5')

Epoch 1/100
2000/2000 [==============================] - 968s 484ms/step - loss: 0.2802 - mean_absolute_error: 0.1892

Epoch 00001: saving model to weights.01.hdf5
Epoch 2/100
2000/2000 [==============================] - 971s 485ms/step - loss: 0.2214 - mean_absolute_error: 0.1528

Epoch 00002: saving model to weights.02.hdf5
Epoch 3/100
2000/2000 [==============================] - 978s 489ms/step - loss: 0.1900 - mean_absolute_error: 0.1332

Epoch 00003: saving model to weights.03.hdf5
Epoch 4/100
2000/2000 [==============================] - 967s 484ms/step - loss: 0.1667 - mean_absolute_error: 0.1181

Epoch 00004: saving model to weights.04.hdf5
Epoch 5/100
2000/2000 [==============================] - 978s 489ms/step - loss: 0.1466 - mean_absolute_error: 0.1046

Epoch 00005: saving model to weights.05.hdf5
Epoch 6/100
2000/2000 [==============================] - 951s 475ms/step - loss: 0.1312 - mean_absolute_error: 0.0935

Epoch 00006: saving model to weights.06.hdf5
Epoch 7/100
2000